In [2]:
from energymercato.setup.starting import SpotSetup, RealTimeSetup

creation of specific plant for France


In [3]:
path = "/mnt/g/Mon Drive/SEM/energymercato/"
spot = SpotSetup(path, solo=False)

path  /mnt/g/Mon Drive/SEM is existing ? True


In [4]:
# Setup game
spot.generate_players()
spot.simulate_game_data()

defining players...
Power consumption is taken on Pmax = 52361.149999999994 = self.total_power-self.total_power_enr)*.95


In [5]:
spot.change_round_day()

In [6]:
spot.fix_market()

waiting for filling...


In [7]:
spot.read_prev_from_players()

ok for production orders
ok for market orders


In [8]:
spot.compute_scores()

/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["amount"] = -a["mwh_cost"] * a["p"]
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["amount"] = -a["mwh_cost"] * a["p"]
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [9]:
spot.get_scores()

In [14]:
spot.fix_market()

Let's play for SPOT !

In [10]:
realtime = RealTimeSetup(spot)

ready for next phase


In [11]:
realtime.start_spot()

 Welcome in the SPOT Market of day 2022-01-16 at 0h !
Real consumption = 36664
Real Production = 15311
Balance = 21353
Go to spot file and fill spot column with adjustment to reach balance = 0
Ready for balancing
 Difference of Real Consumption with SPOT is 21353


In [32]:
realtime.read_spot()

Balance was 941. It is now 4043
Balance is not adjusted


/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:280: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj.loc[dj.p > dj.p_max,"p"] = dj.loc[dj.p > dj.p_max,"p_max"].copy()
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:281: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj.loc[dj.p < dj.p_min,"p"] = dj.loc[dj.p < dj.p_min,"p_min"].copy()
/home/projects/energymercato/energymercato/src/energymercato/usecases/usecases.py:282: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [ ]:
realtime.wait_for_adjustment()

In [33]:
realtime.compute_score()

AttributeError: 'DataFrame' object has no attribute 'spot_changes'

In [23]:
spot.get_scores()